# 비트코인 데이터 가져오기

- 3주차에서 실습으로 진행했던 코드입니다
- 업비트 API를 이용하여 비트코인 데이터를 가져와 변수 `df`에 저장합시다

In [1]:
import time
import requests
import pandas as pd
import json

datetimes = []
for year in range(2018,2025):
    datetimes.append(f"{year}-01-01T00%3A00%3A00%2B09%3A00")
datetimes

headers = {"accept": "application/json"}
dfs = []
for datetime in datetimes:
    url = f"https://api.upbit.com/v1/candles/weeks?market=KRW-BTC&to={datetime}&count=53"
    response = requests.get(url, headers=headers)
    time.sleep(0.08)
    dfs.append(pd.DataFrame(json.loads(response.text)))

df = pd.concat(dfs)

df = df.drop(columns=["candle_date_time_utc", "timestamp", "first_day_of_period"])
df.columns = [
    "market",
    "datetime",
    "open",
    "high",
    "low",
    "close",
    "trade_price",
    "trade_volume",
]

df = df.sort_values(by="datetime").reset_index(drop=True)
df = df.drop_duplicates()
df

,market,datetime,open,high,low,close,trade_price,trade_volume
0,KRW-BTC,2017-09-25T09:00:00,4201000.0,4978000.0,4175000.0,4962000.0,1.677939e+09,374.805189
1,KRW-BTC,2017-10-02T09:00:00,4962000.0,5263000.0,4811000.0,5217000.0,1.163689e+09,231.455318
2,KRW-BTC,2017-10-09T09:00:00,5222000.0,6950000.0,5188000.0,6381000.0,4.960136e+08,82.704757
3,KRW-BTC,2017-10-16T09:00:00,6403000.0,7253000.0,5876000.0,7039000.0,6.822296e+08,106.952135
4,KRW-BTC,2017-10-23T09:00:00,7017000.0,7100000.0,6212000.0,6961000.0,1.436444e+10,2141.265727
...,...,...,...,...,...,...,...,...
327,KRW-BTC,2023-11-27T09:00:00,50210000.0,53864000.0,49500000.0,53700000.0,1.122036e+12,21904.936937
328,KRW-BTC,2023-12-04T09:00:00,53761000.0,61312000.0,53690000.0,59835000.0,2.925854e+12,49907.250404
329,KRW-BTC,2023-12-11T09:00:00,59834000.0,59924000.0,55460000.0,56710000.0,2.293218e+12,39598.394123
330,KRW-BTC,2023-12-18T09:00:00,56710000.0,60419000.0,55002000.0,58002000.0,2.082050e+12,35699.226661


# 변화율 컬럼 만들기

- 종가와 시작가의 차를 계산 후 백분율을 구합니다

In [2]:
df["change_rate"] = (df["close"] - df["open"]) / df["open"] * 100
df

,market,datetime,open,high,low,close,trade_price,trade_volume,change_rate
0,KRW-BTC,2017-09-25T09:00:00,4201000.0,4978000.0,4175000.0,4962000.0,1.677939e+09,374.805189,18.114735
1,KRW-BTC,2017-10-02T09:00:00,4962000.0,5263000.0,4811000.0,5217000.0,1.163689e+09,231.455318,5.139057
2,KRW-BTC,2017-10-09T09:00:00,5222000.0,6950000.0,5188000.0,6381000.0,4.960136e+08,82.704757,22.194561
3,KRW-BTC,2017-10-16T09:00:00,6403000.0,7253000.0,5876000.0,7039000.0,6.822296e+08,106.952135,9.932844
4,KRW-BTC,2017-10-23T09:00:00,7017000.0,7100000.0,6212000.0,6961000.0,1.436444e+10,2141.265727,-0.798062
...,...,...,...,...,...,...,...,...,...
327,KRW-BTC,2023-11-27T09:00:00,50210000.0,53864000.0,49500000.0,53700000.0,1.122036e+12,21904.936937,6.950807
328,KRW-BTC,2023-12-04T09:00:00,53761000.0,61312000.0,53690000.0,59835000.0,2.925854e+12,49907.250404,11.298153
329,KRW-BTC,2023-12-11T09:00:00,59834000.0,59924000.0,55460000.0,56710000.0,2.293218e+12,39598.394123,-5.221112
330,KRW-BTC,2023-12-18T09:00:00,56710000.0,60419000.0,55002000.0,58002000.0,2.082050e+12,35699.226661,2.278258


- plotly를 사용하여 백분율을 시각화 합니다

In [3]:
import plotly.express as px
fig = px.line(df, x="datetime", y="change_rate", title='KRW-BTC close price')
fig

- 너무 지그재그 형태로 되어 있어 상승 추세를 가지는지 하락 추세를 가지는지 판단하기 힘들어 보입니다

# 변화율의 이동평균 시각화

- 한달, 반년, 1년간의 변화율 이동평균을 구하여 추세를 확인해봅니다
- 이동평균 구하는 코드: `Series.rolling(window=window_size).mean()`
- `window_size`는 몇개의 행을 참고하여 이동평균을 구할 것인지에 대한 값입니다
- 만약 `window_size`가 4라면 4개의 행, 즉 4주간의 변화율을 참고하여 이동평균을 계산하는 것입니다

In [4]:
window_size = [4, 26, 52]
for ws in window_size:
    df[f"change_rate_ma_{ws}"] = df["change_rate"].rolling(window=ws).mean()
df.head(30)

,market,datetime,open,high,low,close,trade_price,trade_volume,change_rate,change_rate_ma_4,change_rate_ma_26,change_rate_ma_52
0,KRW-BTC,2017-09-25T09:00:00,4201000.0,4978000.0,4175000.0,4962000.0,1.677939e+09,374.805189,18.114735,NaN,NaN,NaN
1,KRW-BTC,2017-10-02T09:00:00,4962000.0,5263000.0,4811000.0,5217000.0,1.163689e+09,231.455318,5.139057,NaN,NaN,NaN
2,KRW-BTC,2017-10-09T09:00:00,5222000.0,6950000.0,5188000.0,6381000.0,4.960136e+08,82.704757,22.194561,NaN,NaN,NaN
3,KRW-BTC,2017-10-16T09:00:00,6403000.0,7253000.0,5876000.0,7039000.0,6.822296e+08,106.952135,9.932844,13.845299,NaN,NaN
4,KRW-BTC,2017-10-23T09:00:00,7017000.0,7100000.0,6212000.0,6961000.0,1.436444e+10,2141.265727,-0.798062,9.117100,NaN,NaN
5,KRW-BTC,2017-10-30T09:00:00,6961000.0,8620000.0,6741000.0,8453000.0,5.925991e+10,7564.025330,21.433702,13.190761,NaN,NaN
6,KRW-BTC,2017-11-06T09:00:00,8454000.0,8639000.0,6476000.0,7132000.0,1.835143e+11,23820.757797,-15.637568,3.732729,NaN,NaN
7,KRW-BTC,2017-11-13T09:00:00,7132000.0,8899000.0,6921000.0,8770000.0,4.415181e+11,54083.183703,22.966910,6.991245,NaN,NaN
8,KRW-BTC,2017-11-20T09:00:00,8770000.0,10750000.0,7182000.0,10344000.0,1.035080e+12,111389.109831,17.947548,11.677648,NaN,NaN
9,KRW-BTC,2017-11-27T09:00:00,10339000.0,13750000.0,10017000.0,13128000.0,2.704481e+12,223065.970932,26.975530,13.063105,NaN,NaN


In [5]:
for ws in window_size:
    fig = px.line(df, x="datetime", y=f"change_rate_ma_{ws}", title=f'KRW-BTC Change rate week {ws}')
    fig.show()

- 변화율 이동평균을 구하여 시각화 하니 비트코인의 추세를 확인하기 훨씬 더 편해졌습니다

# 상승, 하락한 캔들 세어보기

- 조건 검색을 사용하여 변화율이 상승한 주, 같은 주, 하락한 주의 수를 가져옵니다
- `bull_week_df`: 변화율이 상승한 주
- `same_week_df`: 변화율이 같은 주
- `bear_week_df`: 변화율이 하락한 주

In [6]:
bull_week_df = df[df["change_rate"] > 0]
same_week_df = df[df["change_rate"] == 0]
bear_week_df = df[df["change_rate"] < 0]

print(f"변화율이 상승한 주의 수: {len(bull_week_df)}")
print(f"변화율이 같은 주의 수: {len(same_week_df)}")
print(f"변화율이 하락한 주의 수: {len(bear_week_df)}")

변화율이 상승한 주의 수: 179
변화율이 같은 주의 수: 0
변화율이 하락한 주의 수: 148


# 상승 주 분석

- 상승한 주를 내림차순 정렬하여 정말 많이 상승했을 때에는 얼만큼 상승했는지 확인해봅니다

In [7]:
bull_week_df.sort_values(by="change_rate", ascending=False).head(10)

,market,datetime,open,high,low,close,trade_price,trade_volume,change_rate,change_rate_ma_4,change_rate_ma_26,change_rate_ma_52
10,KRW-BTC,2017-12-04T09:00:00,13120000.0,24967000.0,13119000.0,17661000.0,7.082129e+12,400329.783096,34.611280,25.625317,NaN,NaN
14,KRW-BTC,2018-01-01T09:00:00,19278000.0,28885000.0,18290000.0,25859000.0,6.721143e+12,295090.024592,34.137359,11.498317,NaN,NaN
20,KRW-BTC,2018-02-12T09:00:00,9319000.0,12755000.0,9305000.0,11966000.0,1.927588e+12,170929.654084,28.404335,-2.253176,NaN,NaN
172,KRW-BTC,2020-12-28T09:00:00,29347000.0,39453000.0,29028000.0,37537000.0,2.788128e+12,82386.619934,27.907452,16.196601,5.200638,3.306113
290,KRW-BTC,2023-03-13T09:00:00,29190000.0,37526000.0,28855000.0,37233000.0,2.753693e+12,81998.303310,27.553957,5.075685,1.538399,-0.242685
9,KRW-BTC,2017-11-27T09:00:00,10339000.0,13750000.0,10017000.0,13128000.0,2.704481e+12,223065.970932,26.975530,13.063105,NaN,NaN
80,KRW-BTC,2019-04-01T09:00:00,4661000.0,5975000.0,4621000.0,5910000.0,6.246479e+11,115235.813522,26.796825,8.523015,-0.385584,-0.076829
179,KRW-BTC,2021-02-08T09:00:00,41921000.0,53380000.0,41562000.0,52706000.0,4.656545e+12,94981.312893,25.726963,8.301167,5.735650,3.429027
170,KRW-BTC,2020-12-14T09:00:00,20786000.0,26562000.0,20610000.0,25880000.0,1.329555e+12,55662.505770,24.506880,6.752702,3.478418,2.462765
180,KRW-BTC,2021-02-15T09:00:00,52711000.0,65985000.0,50313000.0,65417000.0,4.654387e+12,79123.025896,24.105026,16.854866,6.686858,3.877080


# 하락 주 분석

- 하락한 주를 오름차순 정렬하여 정말 많이 하락했을 때에는 얼만큼 하락했는지 확인해봅니다

In [8]:
bear_week_df.sort_values(by="change_rate").head(10)

,market,datetime,open,high,low,close,trade_price,trade_volume,change_rate,change_rate_ma_4,change_rate_ma_26,change_rate_ma_52
18,KRW-BTC,2018-01-29T09:00:00,13351000.0,13489000.0,7830000.0,8900000.0,1.840091e+12,178656.161715,-33.338327,-22.913455,NaN,NaN
130,KRW-BTC,2020-03-09T09:00:00,9883000.0,10064000.0,5489000.0,6909000.0,8.926681e+11,119969.765697,-30.092077,-11.418104,-1.661109,1.372382
60,KRW-BTC,2018-11-19T09:00:00,6431000.0,6436000.0,4126000.0,4616000.0,3.804459e+11,74525.100185,-28.222671,-9.912248,-1.796282,-0.508919
193,KRW-BTC,2021-05-17T09:00:00,57655000.0,58000000.0,39331000.0,42585000.0,8.457180e+12,170176.249978,-26.138236,-6.671093,4.024526,3.340794
15,KRW-BTC,2018-01-08T09:00:00,25859000.0,26449000.0,14104000.0,19206000.0,6.374470e+12,296071.182295,-25.727986,-0.907294,NaN,NaN
16,KRW-BTC,2018-01-15T09:00:00,19210000.0,22858000.0,11543000.0,14640000.0,4.899876e+12,316043.448680,-23.789693,-4.239434,NaN,NaN
271,KRW-BTC,2022-11-07T09:00:00,29470000.0,29658000.0,22500000.0,22510000.0,2.377020e+12,94560.012244,-23.617238,-4.292098,-1.923948,-2.097300
250,KRW-BTC,2022-06-13T09:00:00,34718000.0,35050000.0,23800000.0,26642000.0,3.133205e+12,111832.911632,-23.261709,-8.584048,-2.661225,-0.542738
26,KRW-BTC,2018-03-26T09:00:00,9480000.0,9570000.0,7087000.0,7482000.0,8.779568e+11,107151.896124,-21.075949,-11.677610,3.519066,NaN
189,KRW-BTC,2021-04-19T09:00:00,75692000.0,76829000.0,54964000.0,59859000.0,6.241129e+12,96745.941381,-20.917666,-2.058623,6.370343,4.139167


# 희망 회로 돌려보기

- 매 주 시작가에 구매하여 최고가에 팔았을 때 얼만큼 수익을 챙길 수 있는지 희망회로도 돌려봅시다

In [9]:
df["han_river_view_rate"] = (df["high"] - df["open"]) / df["open"] * 100
fig = px.line(df, x="datetime", y="han_river_view_rate", title='Plz....')
fig

# 절망 회로 돌려보기

- 매 주 시작가에 구매하여 최저가에 팔았을 때 얼만큼 잃을 수 있는지 절망회로도 돌려봅시다

In [10]:
df["han_river_go_rate"] = (df["low"] - df["open"]) / df["open"] * 100
fig = px.line(df, x="datetime", y="han_river_go_rate", title='Oh no.....')
fig

# 희망 회로, 절망 회로 비교해보기

- 희망 회로, 절망 회로 역시 추세를 확인하기 힘드네요
- 이번에도 이동평균을 적용하여 추세를 확인해봅시다

In [11]:
import plotly.graph_objects as go

for ws in window_size:
    df[f"han_river_view_rate_ma_{ws}"] = df["han_river_view_rate"].rolling(window=ws).mean()
    df[f"han_river_go_rate_ma_{ws}"] = df["han_river_go_rate"].rolling(window=ws).mean()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["datetime"], y=df[f"han_river_view_rate_ma_{ws}"], name="han_river_view",
                line=dict(color="firebrick", width=1)))

    fig.add_trace(go.Scatter(x=df["datetime"], y=df[f"han_river_go_rate_ma_{ws}"], name="han_river_go",
                line=dict(color="royalblue", width=1)))
    fig.update_layout(title_text=f"KRW-BTC {ws} weeks hope, depair rate")
    fig.show()

- 하락추세가 마치 상승추세를 뒤집어서 오른쪽으로 이동 시켜놓은 것 같습니다
- 상승 이후에는 하락이 필연적으로 온다라는 뜻이겠죠

# 최고가, 최저가 변동률 알아보기

- 한 주의 최대 변동률을 계산해봅니다

In [12]:
df["high_low_change_rate"] = (df["high"] - df["low"]) / df["low"] * 100
fig = px.line(df, x="datetime", y="high_low_change_rate", title='KRW-BTC high_low_change_rate')
fig

- 이번에도 이동평균을 적용하여 추세를 확인해봅시다

In [13]:
window_size = [4, 26, 52]
for ws in window_size:
    df[f"high_low_change_rate_{ws}"] = df["high_low_change_rate"].rolling(window=ws).mean()
    fig = px.line(df, x="datetime", y=f"high_low_change_rate_{ws}", title=f'KRW-BTC high_low_change_rate {ws} week')
    fig.show()